In [1]:
import pandas as pd
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests

import torch
import torch.nn.functional as F

import itertools
from tabulate import tabulate



from unit import node_name2vec

/Users/naokiyoshimaru/opt/anaconda3/envs/FlavorGraph_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("classification_data/node_classification_160.csv")
cd = pd.read_csv("classification_data/node_classification_hub.csv")
categories = cd.columns

In [3]:
food_ls = data["ingredient"]
food_ls = [food.replace("_"," ") for food in food_ls]

In [118]:
from googletrans import Translator

tr = Translator()
trans_food_ls = []
for txt in food_ls:
    result = tr.translate(txt, src="en", dest="ja").text
    trans_food_ls.append(result)
data["trans_ingredient"] = trans_food_ls

In [119]:
ptn_1 = '(?<=>).+?(?=<)'

col_ls = ["Kingdom","Clade1","Clade2","Clade3","Clade4","Order","Family","Genus","Species"]
add_df = pd.DataFrame(columns=col_ls)

for i, food_i in tqdm(enumerate(data["ingredient"])):
    food_i = food_i.replace("_"," ")
    url_i = 'https://en.wikipedia.org/wiki/{}'.format(food_i)
    req_i = requests.get(url_i)
    soup_i = BeautifulSoup(req_i.content, 'html.parser')
    add_ls = []
    for n in range(5,13+1):
        try:
            trg_val = str(soup_i.select('#mw-content-text > div.mw-parser-output > table.infobox.biota > tbody > tr:nth-child({}) > td:nth-child(2) > a'.format(n))[0])
            add_ls.append(re.findall(ptn_1,trg_val))
        except:
            add_ls.append("")
    add_df.loc[i] = add_ls
data = pd.concat([data, add_df], axis=1)

160it [02:13,  1.19it/s]


In [122]:
# data.to_csv("node_classification_160_wikiCategory.csv",index=0)


In [4]:
data = pd.read_csv("classification_data_adv/node_classification_160_wikiCategory.csv")

def cal_cosAVG(file,category,trg_category):
    only_category_data = data[data["category"] == category]
    only_category_LS = list(only_category_data.groupby(trg_category).count()["ingredient"].index)
    rtn_ls = []
    for detal_ct in only_category_LS:
        
        trg_df = only_category_data[only_category_data [trg_category] == detal_ct]
        if len(trg_df) <= 1:
            pass
        else:
            food_pair = list(itertools.permutations(trg_df["ingredient"],2))
            sum_cos = 0
            
            for pair_i in food_pair:
                a = torch.FloatTensor(node_name2vec(pair_i[0],file))
                b = torch.FloatTensor(node_name2vec(pair_i[1],file))
                cos_val = F.cosine_similarity(a, b, dim=0)
                sum_cos += float(cos_val)
            # print("{}({}):".format(detal_ct,len(trg_df)),end="")
            avg_i = sum_cos/len(food_pair)
            rtn_ls.append(avg_i)
    return rtn_ls
    print("========================")

In [23]:
def create_category_ls(ctg,trg_ctg):
    only_category_data = data[data["category"] == ctg]
    only_category_LS = list(only_category_data.groupby(trg_ctg).count()["ingredient"].index)
    only_category_LS = [i + "({})".format(count_row(only_category_data,trg_ctg,i)) for i in only_category_LS]
    return only_category_LS
def count_row(df,col,ctg):
    count_num = df[df[col] == ctg]
    return len(count_num)

def create_res_table(ctg,trg_ctg):
    res_matrix_headers = ["method"]
    res_matrix_headers.extend(create_category_ls(ctg,trg_ctg))
    res_matrix = []
    for f in comp_file_ls:
        add_ls = [f.replace(".pickle","").replace("embedding_data/","")]
        add_ls.extend(cal_cosAVG(f,ctg,trg_ctg))
        res_matrix.append(add_ls)
    result=tabulate(res_matrix, res_matrix_headers, tablefmt="grid")
    print(result)
    

メモ
界→門→綱→目→科→属→種

In [24]:
comp_file_ls = ["embedding_data/Metapath2vec_8212.pickle","embedding_data/Metapath2vec+CSP_8212.pickle","embedding_data/Metapath2vec+NPMI_8212.pickle","embedding_data/Metapath2vec+M2V.pickle","embedding_data/Metapath2vec+NPMI+M2V.pickle"]
comp_file_ls = ["embedding_data/Metapath2vec_8212.pickle","embedding_data/Metapath2vec+CSP_8212.pickle","embedding_data/Metapath2vec+M2V_weighted.pickle","embedding_data/Metapath2vec+NPMI+M2V_weighted.pickle"]
comp_file_ls = ["embedding_data/Metapath2vec.pickle","embedding_data/Metapath2vec+CSP.pickle"]


In [25]:
create_res_table("Seafood","Clade1")

+------------------+-----------------+---------------+---------------+
| method           |   Arthropoda(5) |   Chordata(8) |   Mollusca(6) |
+==================+=================+===============+===============+
| Metapath2vec     |        0.227404 |      0.184539 |      0.134828 |
+------------------+-----------------+---------------+---------------+
| Metapath2vec+CSP |        0.362868 |      0.349068 |      0.302886 |
+------------------+-----------------+---------------+---------------+


In [26]:
create_res_table("Plant/Vegetable","Order")

+------------------+--------------------+---------------------+--------------+-------------------+
| method           |   Amaranthaceae(1) |   Amaryllidaceae(1) |   Apiales(5) |   Asparagaceae(2) |
+==================+====================+=====================+==============+===================+
| Metapath2vec     |           0.126404 |           0.0338904 |     0.114588 |          0.136324 |
+------------------+--------------------+---------------------+--------------+-------------------+
| Metapath2vec+CSP |           0.322097 |           0.248813  |     0.312423 |          0.302182 |
+------------------+--------------------+---------------------+--------------+-------------------+


In [27]:
create_res_table("Plant/Vegetable","Clade4")

+------------------+------------------+----------------+---------------------+
| method           |   Asparagales(3) |   Asterids(10) |   Caryophyllales(1) |
+==================+==================+================+=====================+
| Metapath2vec     |        0.0297247 |      0.0820181 |           0.0457285 |
+------------------+------------------+----------------+---------------------+
| Metapath2vec+CSP |        0.256853  |      0.269473  |           0.263839  |
+------------------+------------------+----------------+---------------------+


In [28]:
create_res_table("Fruit","Clade3")

+------------------+----------------+---------------+
| method           |   Eudicots(17) |   Monocots(3) |
+==================+================+===============+
| Metapath2vec     |      0.0854147 |      0.114437 |
+------------------+----------------+---------------+
| Metapath2vec+CSP |      0.275211  |      0.293357 |
+------------------+----------------+---------------+


In [29]:
create_res_table("Fruit","Clade4")

+------------------+---------------+------------------+--------------+
| method           |   Asterids(3) |   Commelinids(3) |   Rosids(14) |
+==================+===============+==================+==============+
| Metapath2vec     |     0.0312105 |         0.114437 |    0.0865651 |
+------------------+---------------+------------------+--------------+
| Metapath2vec+CSP |     0.244508  |         0.293357 |    0.27427   |
+------------------+---------------+------------------+--------------+


In [30]:
create_res_table("Fruit","Order")

+------------------+---------------+------------------+-------------------+---------------+---------------+
| method           |   Arecales(1) |   Brassicales(1) |   Cucurbitales(1) |   Ericales(3) |   Myrtales(2) |
+==================+===============+==================+===================+===============+===============+
| Metapath2vec     |     0.0312105 |        0.0409995 |          0.173072 |      0.156124 |     0.0536694 |
+------------------+---------------+------------------+-------------------+---------------+---------------+
| Metapath2vec+CSP |     0.244508  |        0.248578  |          0.357128 |      0.343648 |     0.246771  |
+------------------+---------------+------------------+-------------------+---------------+---------------+


In [31]:
create_res_table("Fruit","Family")

+------------------+----------------+--------------------+----------------+-------------------+
| method           |   Actinidia(1) |   Anacardiaceae(1) |   Arecaceae(1) |   Bromeliaceae(1) |
+==================+================+====================+================+===================+
| Metapath2vec     |      0.0658397 |           0.173072 |       0.186152 |         0.0536694 |
+------------------+----------------+--------------------+----------------+-------------------+
| Metapath2vec+CSP |      0.251058  |           0.357128 |       0.369738 |         0.246771  |
+------------------+----------------+--------------------+----------------+-------------------+
